In [1]:
from langchain_openai import OpenAI

In [5]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

True

In [6]:
llm = OpenAI()
print(llm.invoke('농담을 해봐.'))



아저씨가 삼촌이야? 그럼 아저씨는 삼촌표 간장을 만들어도 맛있겠다!


In [8]:
from langchain import PromptTemplate, LLMChain

template = """문장: {sentence}
{language}로 번역:"""
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

print(prompt.format(sentence = "탁자 위에 고양이가 있다", language = "영어"))

문장: 탁자 위에 고양이가 있다
영어로 번역:


In [9]:
# LLMChain 초기화
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Inference 실행
result = llm_chain.run({"sentence": "탁자 위에 고양이가 있다", "language": "영어"})
print(result)

/tmp/ipykernel_1398368/1897775562.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipykernel_1398368/1897775562.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm_chain.run({"sentence": "탁자 위에 고양이가 있다", "language": "영어"})


There is a cat on the table


# 데이터 연결

## 문서 로드

In [10]:
import csv

# Sample data
data = [
    ['이름', '나이', '도시'],
    ['존', 25, '뉴욕'],
    ['에밀리', 28, '로스엔젤레스'],
    ['미카엘', 22, '시카고']
]

# File name
file_name = 'sample.csv'

# 데이터를 CSV 파일에 기록
with open(file_name, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(data)

print(f'예제 CSV 파일 "{file_name}"를 만들었습니다.')

예제 CSV 파일 "sample.csv"를 만들었습니다.


In [11]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='sample.csv')
data = loader.load()
print(data)

[Document(metadata={'source': 'sample.csv', 'row': 0}, page_content='이름: 존\n나이: 25\n도시: 뉴욕'), Document(metadata={'source': 'sample.csv', 'row': 1}, page_content='이름: 에밀리\n나이: 28\n도시: 로스엔젤레스'), Document(metadata={'source': 'sample.csv', 'row': 2}, page_content='이름: 미카엘\n나이: 22\n도시: 시카고')]


## 문서 분할

In [12]:
# 산과 자연에 대한 샘플 문장
content = """고요한 풍경 속에서 우뚝 솟은 산들은 자연의 아름다움을 지키는 장엄한 수호자처럼 서 있습니다.
청량한 산 공기는 고요함의 속삭임을 전해주며, 바스락거리는 잎사귀들은 야생의 교향곡을 작곡합니다.
자연의 팔레트는 산을 초록과 갈색의 색조로 칠해 경이로운 광경을 만들어냅니다.
해가 뜨면, 황금빛 광채가 산 정상에 비치며, 손길 닿지 않은 야생의 세계를 비춥니다."""

# 파일명
file_name = 'mountain.txt'

# 텍스트 파일에 내용 쓰기
with open(file_name, 'w') as txtfile:
    txtfile.write(content)

In [13]:
with open('mountain.txt') as f:
    mountain = f.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len
)

texts = text_splitter.create_documents([mountain])
print(texts[0])
print(texts[1])
print(texts[2])

page_content='고요한 풍경 속에서 우뚝 솟은 산들은 자연의 아름다움을 지키는 장엄한 수호자처럼 서 있습니다.'
page_content='청량한 산 공기는 고요함의 속삭임을 전해주며, 바스락거리는 잎사귀들은 야생의 교향곡을 작곡합니다.
자연의 팔레트는 산을 초록과 갈색의 색조로 칠해 경이로운 광경을 만들어냅니다.'
page_content='해가 뜨면, 황금빛 광채가 산 정상에 비치며, 손길 닿지 않은 야생의 세계를 비춥니다.'


## 텍스트 임베딩

In [14]:
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

embeddings_model = OpenAIEmbeddings(model ='text-embedding-3-small')

embeddings = embeddings_model.embed_documents(
    [
        "Good morning!",
        "Oh, hello!",
        "I want to report an accident",
        "Sorry to hear that. May I ask your name?",
        "Sure, Mario Rossi."
    ]
)

print("임베드된 문서:")
print(f"Number of vector: {len(embeddings)}; Dimension of each vector: {len(embeddings[0])}")

embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")

print("임베드 질의:")
print(f"Dimension of the vector: {len(embedded_query)}")
print(f"Sample of the first 5 elements of the vector: {embedded_query[:5]}")

임베드된 문서:
Number of vector: 5; Dimension of each vector: 1536
임베드 질의:
Dimension of the vector: 1536
Sample of the first 5 elements of the vector: [-0.010684116743505001, -0.010173137299716473, -0.0019674645736813545, 0.023056013509631157, -0.02686513401567936]


In [15]:
# 대화를 txt 파일에 저장
# 대화 행 목록
dialogue_lines = [
    "Good morning!",
    "Oh, hello!",
    "I want to report an accident",
    "Sorry to hear that. May I ask your name?",
    "Sure, Mario Rossi."
]

# 파일명
file_name = 'dialogue.txt'

# 대화 행들을 텍스트 파일에 기록
with open(file_name, 'w') as txtfile:
    for line in dialogue_lines:
        txtfile.write(line + '\n')

print(f'대화 텍스트 파일 "{file_name}"를 만들었습니다.')

대화 텍스트 파일 "dialogue.txt"를 만들었습니다.


## 벡터 스토어

In [17]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 67.2 MB/s eta 0:00:0000:0100:01


In [18]:
from langchain.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

from dotenv import load_dotenv
load_dotenv()

# 문서를 로드해 청크들로 분할하고, 각 청크를 임베드해 벡터 스토어에 로드
raw_documents = TextLoader('dialogue.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0, separator = "\n",)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

# 고밀도 벡터의 효율적인 유사도 검색 및 클러스터링을 위해 메타 AI 리서치에서 개발한 FAISS 활용

In [19]:
query = "What is the reason for calling?"
docs = db.similarity_search(query)
print(docs[0].page_content)

I want to report an accident


In [20]:
print(documents[2])

page_content='Sorry to hear that. May I ask your name?' metadata={'source': 'dialogue.txt'}


## 벡터 저장소에 검색기 '마운트'

In [21]:
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI

retriever = db.as_retriever()

In [29]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

query = "What was the reason of the call?"
output = qa.invoke(query)
output['result']

' The reason of the call was to report an accident.'

## 기억 (메모리)

In [34]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import OpenAI

memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
memory.save_context({"input": "안녕하세요, AI에 관한 에세이를 쓸 아이디어를 찾고 있어요"}, {"output": "안녕하세요, LLM에 관해 써보면 어떨까요?"})

memory.load_memory_variables({})

{'history': '\nThe human is looking for ideas to write an essay about AI. The AI suggests writing about LLM.'}

In [31]:
ConversationSummaryMemory.save_context?

Signature:
ConversationSummaryMemory.save_context(
    self,
    inputs: 'Dict[str, Any]',
    outputs: 'Dict[str, str]',
) -> 'None'
Docstring: Save context from this conversation to buffer.
File:      ~/conda/envs/py310/lib/python3.10/site-packages/langchain/memory/summary.py
Type:      function

## 체인

체인(chain)은 일련의 작동과 LLM 호출을 미리 정해둔 것

### LLMChain

가장 일반적인 유형의 체인.  
프롬프트 템플릿, LLM, 출력 파서(옵션)로 구성된다.

In [36]:
# Simple Chain
from langchain_openai import OpenAI
from langchain import PromptTemplate, LLMChain

template = """문장: {sentence}
{language}로 번역:"""
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

llm = OpenAI(temperature=0)

llm_chain = LLMChain(prompt=prompt, llm=llm)

llm_chain.predict(sentence="탁자 위에 고양이가 있어요", language="영어")

' There is a cat on the table.'

In [37]:
# LLMChain deprecation 해결

from langchain_openai import OpenAI
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnablePassthrough

template = """문장: {sentence}
{language}로 번역:"""
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

llm = OpenAI(temperature=0)

output_parser = StrOutputParser()

chain = RunnableSequence(
    {
        "sentence": RunnablePassthrough(),
        "language": RunnablePassthrough()
    }
    | prompt
    | llm
    | output_parser
)

result = chain.invoke({
    "sentence": "탁자 위에 고양이가 있어요",
    "language": "영어"
})
print(result)


There is a cat on the table.


### RouterChain

특정 조건에 따라 입력 변수를 다른 체인으로 라우팅할 수 있는 체인 유형

In [38]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

llm = OpenAI()

itinerary_template = """당신은 휴가 일정 조수입니다. \
당신은 고객이 최고의 목적지와 일정을 찾도록 도와줍니다. \
당신은 고객의 선호에 따라 최적화된 일정을 작성하는 데 도움을 줍니다.

여기에 질문이 있습니다:
{input}"""

restaurant_template = """당신은 레스토랑 예약 조수입니다. \
당신은 고객의 손님 수와 음식 선호를 확인합니다. \
특별한 조건을 고려해야 하는지 주의합니다.

여기에 질문이 있습니다:
{input}"""

prompt_infos = [
    {
        "name": "여행 일정",
        "description": "여행 일정 작성을 돕습니다",
        "prompt_template": itinerary_template,
    },
    {
        "name": "레스토랑",
        "description": "고객의 레스토랑 예약을 도와줍니다",
        "prompt_template": restaurant_template,
    },
]

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

/tmp/ipykernel_1398368/347779221.py:45: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  default_chain = ConversationChain(llm=llm, output_key="text")
/home/haiqv/conda/envs/py310/lib/python3.10/site-packages/pydantic/main.py:214: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/tmp/ipykernel_1398368/347779221.py:57: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(


In [39]:
print(chain.invoke("밀라노에서 베니스까지 자동차로 여행하려고 합니다. 중간에 들릴 만한 명소가 있나요?."))



> Entering new MultiPromptChain chain...
여행 일정: {'input': '밀라노에서 베니스까지 자동차로 여행하려고 합니다.'}
> Finished chain.
{'input': '밀라노에서 베니스까지 자동차로 여행하려고 합니다.', 'text': ' 어떤 경로가 가장 좋을까요?\n\n일단 베니스는 이탈리아 북부에 위치하고 있으며, 이탈리아의 주요 도시 중 하나인 밀라노에서는 차로 약 3시간 거리에 있습니다. 따라서 베니스로 가는 가장 간단한 경로는 밀라노에서 A4 고속도로를 타고 동쪽으로 이동하는 것입니다. 이 도로는 밀라노에서 베니스까지 직접 이어져 있으며, 여러 번의 톨 게이트를 지나야 합니다.\n\n하지만 만약 당신이 좀 더 멋진 경험을 원한다면, 다른 루트를 고려해볼 수도 있습니다. 밀라노에서 베니스로 가는 도로 중 가장 유명하고 아름다운 도로는'}


In [40]:
print(chain.invoke("오늘 저녁 식사를 예약하고 싶어요"))



> Entering new MultiPromptChain chain...
레스토랑: {'input': '고객의 레스토랑 예약을 도와줍니다오늘 저녁 식사를 예약하고 싶어요'}
> Finished chain.
{'input': '고객의 레스토랑 예약을 도와줍니다오늘 저녁 식사를 예약하고 싶어요', 'text': '. 몇 명이 식사하실 건가요?\n\n고객님, 몇 분이 예약을 원하시는지 알려주시겠어요? 또한 음식 선호도 있나요? 예를 들어, 알러지나 식이 제한 사항이 있으신가요? 이러한 정보를 알려주시면 더 좋은 서비스를 제공해드릴 수 있어요. '}


### SequentialChain

여러 개의 체인을 순서대로 실행할 수 있는 체인 유형

In [43]:
# 주어진 주제에 대한 농담을 먼저 생성한 다음 -> 이를 다른 언어로 번역하는 AI 시스템
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# 이 코드는 연극의 제목을 주면 시놉시스를 작성하는 LLMChain입니다.
llm = OpenAI(temperature=0)
template = """당신은 코미디언입니다. {topic}에 관한 농담을 생성하세요.
농담:"""
prompt_template = PromptTemplate(input_variables=["topic"], template=template)
joke_chain = LLMChain(llm=llm, prompt=prompt_template)

template = """당신은 번역가입니다. 주어진 텍스트 입력을 {language}로 번역하세요.
번역:"""
prompt_template = PromptTemplate(input_variables=["language"], template=template)
translator_chain = LLMChain(llm=llm, prompt=prompt_template)

print(translator_chain)

verbose=False prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='당신은 번역가입니다. 주어진 텍스트 입력을 {language}로 번역하세요.\n번역:') llm=OpenAI(client=<openai.resources.completions.Completions object at 0x7f14e8b53670>, async_client=<openai.resources.completions.AsyncCompletions object at 0x7f14e89838b0>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')) output_parser=StrOutputParser() llm_kwargs={}


In [44]:
# 이 코드는 두 체인을 순서대로 실행하는 전체 체인입니다.
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[joke_chain, translator_chain], verbose=True)
translated_joke = overall_chain.run("고양이와 개")



> Entering new SimpleSequentialChain chain...
 "고양이와 개가 싸우면 누가 이길까요? 당연히 고양이죠, 개는 꼬리를 쫓는 것밖에 못하니까요!"
 If cats and dogs fight, who do you think will win? Of course, it's the cat. Dogs can only chase their tails!

> Finished chain.


### TransformationChain

일부 함수나 표현식을 사용해 입력 변수나 다른 체인의 출력을 변환할 수 있는 체인 유형

In [45]:
# string 모듈을 임포트합니다
import string

# 함수를 정의합니다
def rename_cat(inputs: dict) -> dict:
    # 파일을 읽기 모드로 엽니다
    text = inputs["text"]
    # 'cat'을 'Silvester the Cat'으로 바꿉니다
    new_text = text.replace('cat', 'Silvester the Cat')
    # 변경된 텍스트를 반환합니다
    return {"output_text": new_text}

In [46]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

with open("Cats&Dogs.txt") as f:
    cats_and_dogs = f.read()

import string

transform_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=rename_cat
)

template = """이 텍스트를 요약하세요:

{output_text}

요약:"""
prompt = PromptTemplate(input_variables=["output_text"], template=template)
llm_chain = LLMChain(llm=OpenAI(), prompt=prompt)

sequential_chain = SimpleSequentialChain(chains=[transform_chain, llm_chain])

sequential_chain.invoke(cats_and_dogs)

{'input': "\nThe Cat and the Dog\n\nThere was once a cat and a dog who lived in the same house. They did not get along very well, as they often fought over food, toys, and attention. The cat was clever and cunning, while the dog was loyal and friendly. The cat liked to tease the dog, and the dog liked to chase the cat.\n\nOne day, the cat decided to play a prank on the dog. He found a ball of yarn and tied it around the dog's tail. Then he hid behind a sofa and waited for the dog to notice. When the dog saw the yarn, he thought it was a toy and started to play with it. He ran around the house, trying to catch the yarn with his mouth. But every time he got close, the yarn moved away from him. The cat laughed silently as he watched the dog's futile attempts.\n\nThe dog soon realized that something was wrong. He looked behind him and saw that the yarn was attached to his tail. He tried to pull it off, but it was too tight. He felt angry and embarrassed. He wondered who did this to him. He

## Agent (에이전트)

에이전트는 LLM 기반 애플리케이션 내에서 의사 결정을 내리는 주체

In [47]:
from langchain import SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain_openai import OpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

search = SerpAPIWrapper()

In [48]:
tools = [Tool.from_function(
        func=search.run,
        name="Search",
        description="현재 이벤트에 관해 질문할 때 유용함"
    )]

agent = initialize_agent(tools, llm = OpenAI(), agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.invoke("아바타 2 개봉일은?")

/tmp/ipykernel_1398368/1685785234.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm = OpenAI(), agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)




> Entering new AgentExecutor chain...
 아바타 2 개봉일을 알고 싶다면, 검색 도구를 사용하는 것이 좋다.
Action: Search
Action Input: "아바타 2 개봉일"
Observation: December 16, 2022
Thought: 검색 결과로 2022년 12월 16일이 나왔다.
Final Answer: December 16, 2022

> Finished chain.


{'input': '아바타 2 개봉일은?', 'output': 'December 16, 2022'}

In [50]:
!pip install langgraph 

In [54]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [55]:
tools = [Tool.from_function(
        func=search.run,
        name="Search",
        description="현재 이벤트에 관해 질문할 때 유용함"
    )]

# agent = initialize_agent(tools, llm = OpenAI(), agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
# agent.invoke("아바타 2 개봉일은?")

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
model = ChatOpenAI(model="gpt-4o", temperature=0)
graph = create_react_agent(model, tools=tools)

inputs = {"messages": [("user", "아바타 2 개봉일은?")]}
print_stream(graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

아바타 2 개봉일은?
================================== Ai Message ==================================
Tool Calls:
  Search (call_5rMEeD6YOjFSk6mw5fzFO1qF)
 Call ID: call_5rMEeD6YOjFSk6mw5fzFO1qF
  Args:
    __arg1: 아바타 2 개봉일
================================= Tool Message =================================
Name: Search

December 16, 2022
================================== Ai Message ==================================

영화 "아바타 2"는 2022년 12월 16일에 개봉되었습니다.
